In [1]:
import math
import numpy as np
import pandas as pd
import random
from mmh3 import hash as hash2
import os

In [2]:
# ! unzip data.csv.zip

In [3]:
df = pd.read_csv('data.csv')

## Moment 0

In [4]:
print(f' ground truth: {len(np.unique(df.host))}')

 ground truth: 135268


In [5]:
def count_zeros(n):
    s = bin(n).replace("0b", "")
    return len(s) - len(s.rstrip('0'))


def flajolet_martin(stream, no_hashes, cats):
    max_zeros = [[0 for i in range(no_hashes)] for j in range(cats)]
    seeds = [[random.randint(1, no_hashes * cats * 100) for i in range(no_hashes)] for j in range(cats)]
    
    for number in stream:
        for i in range(cats):
            for j in range(no_hashes):
                max_zeros[i][j] = max(max_zeros[i][j], count_zeros(hash2(str(number), seeds[i][j], signed = False))) 
    
    list_median = []
    for i in range(0, cats):
        list_median.append(np.mean(max_zeros[i]))
    
    power = np.median(list_median)
    return 2 ** int(power)

In [6]:
for L,G in zip([4, 4, 8 , 8],[2, 1, 2, 4]):
    print(f'L:{L}, G:{G}, {distinct_estimation(df.host, L , G)}')

L:4, G:2, 262144
L:4, G:1, 131072
L:8, G:2, 131072
L:8, G:4, 131072


In [ ]:
for L,G in zip([2, 1, 2, 4],[4, 4, 8 , 8]):
    print(f'L:{L}, G:{G}, {distinct_estimation(df.host, L , G)}')

## Moment 2

In [11]:
def exact_2nd_moment(stream):
    d = {}
    for number in stream:
        d[number] = d.get(number, 0) + 1
    return sum(x ** 2  for x in d.values())

In [12]:
print(f' ground truth: {exact_2nd_moment(df.host)}')

 ground truth: 2385064517


In [10]:
def alon_matias_szegedy(stream, no_blocks, cats):
    d = []
    for i in range(cats):
        d.append([])
    time = 0

    for stream_item in stream:
        time += 1
        for i in range(cats):
            block_items = []
            indexes = []
            
            for count in d[i]:
                if count['el'] == stream_item:
                    count['val'] += 1
                    break
            else:
                if len(d[i]) < no_blocks:
                    d[i].append({'el': stream_item, 'val': 1})
                elif random.random() < no_blocks / time:
                    d[i][random.randint(0, len(d[i]) - 1)] = {'el': stream_item, 'val': 1}

    return int(np.median([np.mean([time * (2 * x['val'] - 1) for x in count]) for count in d]))

In [14]:
alon_matias_szegedy(df.host, 100 , 5)

2463063102


In [ ]:
for L,G in zip([50, 100, 200, 300 , 400],[5, 5, 5, 5, 5]):
    print(f'L:{L}, G:{G}, {alon_matias_szegedy(df.host, L , G)}')
    

In [ ]:
for L,G in zip([100, 100, 100, 100],[5, 10, 20, 50]):
    print(f'L:{L}, G:{G}, {alon_matias_szegedy(df.host, L , G)}')